This file allows to test qudi hardware files for AWGs. To ease sharing with external partners, it doesn't require a full qudi installation, but only a simple mockup plus the actual hardware file. 

WARNING: AWG70k needs credentials, don't send/push to git your credentials with this file!

In [1]:
import os
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import time, datetime

# Set folder of the hardware file
path_driver = r'C:\qudi01\hardware\awg'
# Setup the mockup environment that will simulate qudi
path_mockup = r'C:\Users\IQO\test_scripts_timo\mockup'

sys.path.append(path_mockup)
os.chdir(path_driver)

# import the hardware file
avail_hw = []
try:
    from keysight_m819x import AWGM8190A, AWGM819X
    avail_hw.append("keysight m8190a")
except:
    pass
try:
    from tektronix_awg70k import AWG70K
    avail_hw.append("tek 70k")
except:
    pass

print(f"Found hw files for {avail_hw}")

Found hw files for ['tek 70k']


In [2]:
level = logging.INFO

logging.basicConfig(level=logging.DEBUG)
#logging.basicConfig(filename='logfile.log', filemode='w', level=logging.DEBUG)
logger = logging.getLogger(__name__)
logger.level = logging.DEBUG
print(f"logger {logger.name}: level {logger.level}")

# silence all other logger
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if logger.name != __name__:
        logger.setLevel(logging.INFO)
        #print(f"Set {logger.name} to info")



logger __main__: level 10


# Library methods

In [28]:
def sample_sine(n_samples, f_sa=1e-3):
    y = np.zeros(n_samples, dtype=np.float32)
    t = np.linspace(1, n_samples, n_samples)
    y = 0.9*np.sin(2*np.pi*f_sa*t)
    
    return y

def create_sine_wave(n_samples, channel_activation, f_sa=1e-3):
    """
    Creates analog/digital sample arrays as expected from qudi
    awg hw files. Filled with some data.
    """
    n_samples = int(n_samples)
    
    analog_samples = sample_sine(n_samples, f_sa=1e-2)
    digital_samples = np.zeros(n_samples, dtype=bool)
    
    a_dict = {}
    d_dict = {}
    
    for ch in channel_activation:
        if 'a_' in ch:
            a_dict[ch] = analog_samples
        if 'd_' in ch:
            d_dict[ch] = digital_samples
    
    return a_dict, d_dict

def write_wave(waveform_name, analog_samples, digital_samples):

    n_samples = len(analog_samples['a_ch1'])
    
    t_start = time.perf_counter()
    written_samples, wfm_list = awg.write_waveform(
                                    name=waveform_name,
                                    analog_samples=analog_samples,
                                    digital_samples=digital_samples,
                                    is_first_chunk=True,
                                    is_last_chunk=True,
                                    total_number_of_samples=n_samples)
    t_end = time.perf_counter()
    
    now = datetime.datetime.now()
    dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
    
    for wave in wfm_list:
        wave_files = [f for f in os.listdir(awg._tmp_work_dir) if wave in f]
        try:
            for file in wave_files:
                os.remove(awg._tmp_work_dir + "\\" + file)
        except:
            awg.log.exception(f"Couldn't delete wave from temp dir {wave}")
        
    awg.log.info(f"{dt_string} Wrote wave {waveform_name} ({written_samples} Sa) in {1000*(t_end-t_start):.2f} ms")
    
    return wfm_list

['test1_ch1.wfmx']

# Init AWG

## set essential params for keysight AWG

In [14]:
awg = AWGM8190A(None)
awg.log = logger

awg._visa_address = 'TCPIP0::localhost::hislip0::INSTR'
awg._pulsed_file_dir = ""
awg._assets_storage_path = ""
awg._awg_timeout = 20
awg._dac_resolution = 14
awg._sample_rate_div = 1
awg._d_ch_level_low_high = []
awg._wave_mem_mode = 'awg_segments'


## set params for Tek 70 k

In [4]:
awg = AWG70K(None)
awg.log = logger

awg._visa_address = 'TCPIP0::192.168.1.2::inst0::INSTR'
awg._pulsed_file_dir = ""
awg._assets_storage_path = ""
awg._tmp_work_dir = "C:\\qudi01\\hardware\\awg\\tmp_work_dir"
awg._ip_address = '192.168.1.2'
awg._visa_timeout = 3000
awg._ftp_dir = 'C:\\inetpub\\ftproot' # this path cannot be retrieved otherwise
awg._username = 'OEM'
awg._password = 'XXXX'
awg._default_sample_rate: 50e9


In [32]:
awg._visa_timeout

3000

## Load AWG

In [39]:
awg.on_activate()
activation_full = {ch:True for ch in awg.get_constraints().activation_config['all']}
awg.set_active_channels(activation_full)
ch_activation = awg.get_active_channels()

In [40]:
ch_activation

{'a_ch1': True, 'd_ch2': True, 'd_ch1': True}

### Set log level

In [41]:
awg.log.level=logging.INFO

# Upload test wave

## Sample

In [34]:
n_sa_create = 5*1e6*awg.get_constraints().waveform_length.step
wave_a, wave_d = create_sine_wave(n_sa_create, [ch for (ch, val) in ch_activation.items() if val])
#plt.plot(wave_a['a_ch1'], label='a_ch1')
#plt.plot(wave_d['d_ch1'], label='d_ch1')
#plt.legend()
#plt.show()


In [35]:
len(wave_a['a_ch1'])

10000000

## Upload

In [29]:
wfms = write_wave('test1', wave_a, wave_d)
awg.get_loaded_assets()

INFO:__main__:2021/08/09 08:52:57 Wrote wave test1 (10000000 Sa) in 63391.05 ms


({1: ''}, 'waveform')

## Upload many waves

In [12]:
np.random.randint(50,100)/100

0.64

In [ ]:
n_waveforms = int(1e4)
wave_size_mas = 5
t_start = time.perf_counter()
for i in range(n_waveforms):
    try:
        rand_fac = np.random.randint(50,100)/100
        n_sa_create = int(wave_size_mas*rand_fac*1e6)*awg.get_constraints().waveform_length.step
        wave_a, wave_d = create_sine_wave(n_sa_create, [ch for (ch, val) in ch_activation.items() if val])
        write_wave(f'wave_{i}', wave_a, wave_d)
    except KeyboardInterrupt:
        break
    except:
        awg.log.exception("Upload failed wave_{i}: ")
    
t_end = time.perf_counter()
awg.log.info(f"Wrote {n_waveforms} waves in {t_end-t_start:.2f} s")

INFO:__main__:2021/08/09 09:07:33 Wrote wave wave_0 (6000000 Sa) in 4878.51 ms
INFO:__main__:2021/08/09 09:07:38 Wrote wave wave_1 (5699998 Sa) in 4713.80 ms
INFO:__main__:2021/08/09 09:07:43 Wrote wave wave_2 (5699998 Sa) in 4894.39 ms
INFO:__main__:2021/08/09 09:07:49 Wrote wave wave_3 (7500000 Sa) in 6030.05 ms
INFO:__main__:2021/08/09 09:07:54 Wrote wave wave_4 (5800000 Sa) in 4713.55 ms
INFO:__main__:2021/08/09 09:07:59 Wrote wave wave_5 (6500000 Sa) in 5244.19 ms
INFO:__main__:2021/08/09 09:08:04 Wrote wave wave_6 (6000000 Sa) in 4893.62 ms
INFO:__main__:2021/08/09 09:08:12 Wrote wave wave_7 (8800000 Sa) in 7028.67 ms
INFO:__main__:2021/08/09 09:08:18 Wrote wave wave_8 (8500000 Sa) in 6792.29 ms
INFO:__main__:2021/08/09 09:08:26 Wrote wave wave_9 (9200000 Sa) in 7367.43 ms
INFO:__main__:2021/08/09 09:08:34 Wrote wave wave_10 (9900000 Sa) in 7903.68 ms
INFO:__main__:2021/08/09 09:08:41 Wrote wave wave_11 (9100000 Sa) in 7293.68 ms
INFO:__main__:2021/08/09 09:08:47 Wrote wave wave_

INFO:__main__:2021/08/09 09:18:16 Wrote wave wave_103 (7800000 Sa) in 6288.11 ms
INFO:__main__:2021/08/09 09:18:21 Wrote wave wave_104 (6500000 Sa) in 5303.35 ms
INFO:__main__:2021/08/09 09:18:28 Wrote wave wave_105 (8000000 Sa) in 6678.51 ms
INFO:__main__:2021/08/09 09:18:34 Wrote wave wave_106 (7199998 Sa) in 5821.88 ms
INFO:__main__:2021/08/09 09:18:42 Wrote wave wave_107 (9399998 Sa) in 7656.09 ms
INFO:__main__:2021/08/09 09:18:47 Wrote wave wave_108 (6800000 Sa) in 5496.51 ms
INFO:__main__:2021/08/09 09:18:54 Wrote wave wave_109 (7800000 Sa) in 6277.17 ms
INFO:__main__:2021/08/09 09:19:02 Wrote wave wave_110 (9600000 Sa) in 7703.95 ms
INFO:__main__:2021/08/09 09:19:08 Wrote wave wave_111 (8100000 Sa) in 6558.20 ms
INFO:__main__:2021/08/09 09:19:15 Wrote wave wave_112 (8100000 Sa) in 6530.66 ms
INFO:__main__:2021/08/09 09:19:22 Wrote wave wave_113 (8900000 Sa) in 7297.19 ms
INFO:__main__:2021/08/09 09:19:29 Wrote wave wave_114 (7700000 Sa) in 6353.13 ms
INFO:__main__:2021/08/09 09:

INFO:__main__:2021/08/09 09:28:55 Wrote wave wave_205 (5200000 Sa) in 4349.98 ms
INFO:__main__:2021/08/09 09:29:02 Wrote wave wave_206 (8600000 Sa) in 6963.76 ms
INFO:__main__:2021/08/09 09:29:07 Wrote wave wave_207 (6100000 Sa) in 5060.13 ms
INFO:__main__:2021/08/09 09:29:11 Wrote wave wave_208 (5000000 Sa) in 4075.65 ms
INFO:__main__:2021/08/09 09:29:16 Wrote wave wave_209 (5800000 Sa) in 4970.00 ms
INFO:__main__:2021/08/09 09:29:21 Wrote wave wave_210 (6600000 Sa) in 5378.21 ms
INFO:__main__:2021/08/09 09:29:28 Wrote wave wave_211 (8000000 Sa) in 6473.02 ms
INFO:__main__:2021/08/09 09:29:35 Wrote wave wave_212 (8500000 Sa) in 6828.45 ms
INFO:__main__:2021/08/09 09:29:41 Wrote wave wave_213 (6899998 Sa) in 6077.58 ms
INFO:__main__:2021/08/09 09:29:48 Wrote wave wave_214 (8199998 Sa) in 6619.93 ms
INFO:__main__:2021/08/09 09:29:55 Wrote wave wave_215 (9300000 Sa) in 7472.09 ms
INFO:__main__:2021/08/09 09:30:01 Wrote wave wave_216 (7100000 Sa) in 5909.55 ms
INFO:__main__:2021/08/09 09:

INFO:__main__:2021/08/09 09:39:19 Wrote wave wave_307 (6200000 Sa) in 5096.44 ms
INFO:__main__:2021/08/09 09:39:24 Wrote wave wave_308 (5300000 Sa) in 4445.19 ms
INFO:__main__:2021/08/09 09:39:29 Wrote wave wave_309 (6700000 Sa) in 5478.75 ms
INFO:__main__:2021/08/09 09:39:37 Wrote wave wave_310 (9200000 Sa) in 7439.45 ms
INFO:__main__:2021/08/09 09:39:42 Wrote wave wave_311 (6600000 Sa) in 5430.33 ms
INFO:__main__:2021/08/09 09:39:48 Wrote wave wave_312 (6200000 Sa) in 5100.60 ms
INFO:__main__:2021/08/09 09:39:55 Wrote wave wave_313 (9100000 Sa) in 7450.38 ms
INFO:__main__:2021/08/09 09:40:00 Wrote wave wave_314 (5300000 Sa) in 4399.27 ms
INFO:__main__:2021/08/09 09:40:05 Wrote wave wave_315 (6500000 Sa) in 5516.60 ms
INFO:__main__:2021/08/09 09:40:13 Wrote wave wave_316 (8800000 Sa) in 7297.81 ms
INFO:__main__:2021/08/09 09:40:17 Wrote wave wave_317 (5000000 Sa) in 4256.06 ms
INFO:__main__:2021/08/09 09:40:22 Wrote wave wave_318 (5899998 Sa) in 4994.64 ms
INFO:__main__:2021/08/09 09:

INFO:__main__:2021/08/09 09:48:05 Wrote wave wave_394 (7900000 Sa) in 6389.95 ms
INFO:__main__:2021/08/09 09:48:11 Wrote wave wave_395 (7199998 Sa) in 5853.70 ms
INFO:__main__:2021/08/09 09:48:17 Wrote wave wave_396 (6500000 Sa) in 5307.25 ms
INFO:__main__:2021/08/09 09:48:23 Wrote wave wave_397 (8500000 Sa) in 6890.58 ms
INFO:__main__:2021/08/09 09:48:29 Wrote wave wave_398 (6300000 Sa) in 5257.78 ms
INFO:__main__:2021/08/09 09:48:34 Wrote wave wave_399 (5899998 Sa) in 4794.45 ms
INFO:__main__:2021/08/09 09:48:40 Wrote wave wave_400 (6800000 Sa) in 5848.49 ms
INFO:__main__:2021/08/09 09:48:46 Wrote wave wave_401 (7900000 Sa) in 6398.90 ms
INFO:__main__:2021/08/09 09:48:53 Wrote wave wave_402 (8800000 Sa) in 7114.25 ms
INFO:__main__:2021/08/09 09:48:58 Wrote wave wave_403 (5800000 Sa) in 4771.74 ms
INFO:__main__:2021/08/09 09:49:03 Wrote wave wave_404 (5400000 Sa) in 4460.34 ms
INFO:__main__:2021/08/09 09:49:11 Wrote wave wave_405 (9900000 Sa) in 7989.21 ms
INFO:__main__:2021/08/09 09:

...

INFO:__main__:2021/08/09 11:15:57 Wrote wave wave_1188 (5500000 Sa) in 5891.06 ms
INFO:__main__:2021/08/09 11:16:06 Wrote wave wave_1189 (9000000 Sa) in 8584.16 ms
INFO:__main__:2021/08/09 11:16:14 Wrote wave wave_1190 (8199998 Sa) in 8014.32 ms
INFO:__main__:2021/08/09 11:16:21 Wrote wave wave_1191 (6300000 Sa) in 6541.93 ms
INFO:__main__:2021/08/09 11:16:28 Wrote wave wave_1192 (7700000 Sa) in 7434.40 ms
INFO:__main__:2021/08/09 11:16:37 Wrote wave wave_1193 (8700000 Sa) in 8347.41 ms
INFO:__main__:2021/08/09 11:16:45 Wrote wave wave_1194 (9100000 Sa) in 8630.24 ms
INFO:__main__:2021/08/09 11:16:53 Wrote wave wave_1195 (7700000 Sa) in 7600.12 ms
INFO:__main__:2021/08/09 11:17:00 Wrote wave wave_1196 (6700000 Sa) in 6966.34 ms
INFO:__main__:2021/08/09 11:17:08 Wrote wave wave_1197 (7800000 Sa) in 7434.37 ms
INFO:__main__:2021/08/09 11:17:16 Wrote wave wave_1198 (8600000 Sa) in 8030.03 ms
INFO:__main__:2021/08/09 11:17:25 Wrote wave wave_1199 (8600000 Sa) in 8895.75 ms
INFO:__main__:20

INFO:__main__:2021/08/09 11:29:05 Wrote wave wave_1288 (8900000 Sa) in 9206.87 ms
INFO:__main__:2021/08/09 11:29:11 Wrote wave wave_1289 (5200000 Sa) in 6126.33 ms
INFO:__main__:2021/08/09 11:29:20 Wrote wave wave_1290 (9600000 Sa) in 9226.87 ms
INFO:__main__:2021/08/09 11:29:26 Wrote wave wave_1291 (5699998 Sa) in 6232.68 ms
INFO:__main__:2021/08/09 11:29:33 Wrote wave wave_1292 (5300000 Sa) in 6328.37 ms
INFO:__main__:2021/08/09 11:29:41 Wrote wave wave_1293 (7800000 Sa) in 8331.27 ms
INFO:__main__:2021/08/09 11:29:50 Wrote wave wave_1294 (9000000 Sa) in 8925.14 ms
INFO:__main__:2021/08/09 11:29:58 Wrote wave wave_1295 (7000000 Sa) in 7207.87 ms
INFO:__main__:2021/08/09 11:30:07 Wrote wave wave_1296 (9500000 Sa) in 9365.12 ms
INFO:__main__:2021/08/09 11:30:14 Wrote wave wave_1297 (6800000 Sa) in 7172.52 ms
INFO:__main__:2021/08/09 11:30:22 Wrote wave wave_1298 (6899998 Sa) in 7607.41 ms
INFO:__main__:2021/08/09 11:30:29 Wrote wave wave_1299 (6899998 Sa) in 7151.07 ms
INFO:__main__:20

INFO:__main__:2021/08/09 11:40:57 Wrote wave wave_1377 (8100000 Sa) in 8814.89 ms
INFO:__main__:2021/08/09 11:41:03 Wrote wave wave_1378 (5100000 Sa) in 6320.15 ms
INFO:__main__:2021/08/09 11:41:13 Wrote wave wave_1379 (9500000 Sa) in 9656.27 ms
INFO:__main__:2021/08/09 11:41:20 Wrote wave wave_1380 (6300000 Sa) in 7209.56 ms
INFO:__main__:2021/08/09 11:41:28 Wrote wave wave_1381 (6500000 Sa) in 7346.33 ms
INFO:__main__:2021/08/09 11:41:37 Wrote wave wave_1382 (6600000 Sa) in 8995.82 ms
INFO:__main__:2021/08/09 11:41:47 Wrote wave wave_1383 (9700000 Sa) in 9669.69 ms
INFO:__main__:2021/08/09 11:41:55 Wrote wave wave_1384 (7700000 Sa) in 8198.12 ms
INFO:__main__:2021/08/09 11:42:02 Wrote wave wave_1385 (6000000 Sa) in 6814.03 ms
INFO:__main__:2021/08/09 11:42:11 Wrote wave wave_1386 (8800000 Sa) in 9219.62 ms
INFO:__main__:2021/08/09 11:42:18 Wrote wave wave_1387 (5000000 Sa) in 6484.47 ms
INFO:__main__:2021/08/09 11:42:26 Wrote wave wave_1388 (8500000 Sa) in 8703.15 ms
INFO:__main__:20

INFO:__main__:2021/08/09 11:55:49 Wrote wave wave_1477 (5000000 Sa) in 7232.34 ms
INFO:__main__:2021/08/09 11:55:58 Wrote wave wave_1478 (7199998 Sa) in 8854.22 ms
INFO:__main__:2021/08/09 11:56:05 Wrote wave wave_1479 (5300000 Sa) in 7146.72 ms
INFO:__main__:2021/08/09 11:56:14 Wrote wave wave_1480 (6400000 Sa) in 8619.39 ms
INFO:__main__:2021/08/09 11:56:22 Wrote wave wave_1481 (6700000 Sa) in 8512.74 ms
INFO:__main__:2021/08/09 11:56:33 Wrote wave wave_1482 (9600000 Sa) in 10339.27 ms
INFO:__main__:2021/08/09 11:56:42 Wrote wave wave_1483 (7000000 Sa) in 8635.15 ms
INFO:__main__:2021/08/09 11:56:50 Wrote wave wave_1484 (7500000 Sa) in 8699.44 ms
INFO:__main__:2021/08/09 11:57:01 Wrote wave wave_1485 (5300000 Sa) in 10008.44 ms
INFO:__main__:2021/08/09 11:57:11 Wrote wave wave_1486 (9700000 Sa) in 10504.56 ms
INFO:__main__:2021/08/09 11:57:19 Wrote wave wave_1487 (6200000 Sa) in 8146.28 ms
INFO:__main__:2021/08/09 11:57:28 Wrote wave wave_1488 (5800000 Sa) in 8688.73 ms
INFO:__main__

INFO:__main__:2021/08/09 12:12:17 Wrote wave wave_1577 (5899998 Sa) in 9512.52 ms
INFO:__main__:2021/08/09 12:12:26 Wrote wave wave_1578 (5800000 Sa) in 9172.62 ms
INFO:__main__:2021/08/09 12:12:38 Wrote wave wave_1579 (8700000 Sa) in 11796.94 ms
INFO:__main__:2021/08/09 12:12:48 Wrote wave wave_1580 (5600000 Sa) in 9407.56 ms
INFO:__main__:2021/08/09 12:12:59 Wrote wave wave_1581 (7800000 Sa) in 11780.66 ms
INFO:__main__:2021/08/09 12:13:10 Wrote wave wave_1582 (7199998 Sa) in 10453.36 ms
INFO:__main__:2021/08/09 12:13:24 Wrote wave wave_1583 (9399998 Sa) in 14342.53 ms
INFO:__main__:2021/08/09 12:13:35 Wrote wave wave_1584 (8900000 Sa) in 10933.12 ms
INFO:__main__:2021/08/09 12:13:45 Wrote wave wave_1585 (5300000 Sa) in 8966.78 ms
INFO:__main__:2021/08/09 12:13:55 Wrote wave wave_1586 (6400000 Sa) in 10232.95 ms
INFO:__main__:2021/08/09 12:14:04 Wrote wave wave_1587 (5100000 Sa) in 9195.53 ms
INFO:__main__:2021/08/09 12:14:16 Wrote wave wave_1588 (8400000 Sa) in 11602.09 ms
INFO:__ma

In [37]:
awg._get_filenames_on_device()

['hahn_echo_ch1.wfmx',
 'C13-phase-cal_0_ch1.wfmx',
 'C13-phase-cal_10_ch1.wfmx',
 'C13-phase-cal_11_ch1.wfmx',
 'C13-phase-cal_12_ch1.wfmx',
 'C13-phase-cal_13_ch1.wfmx',
 'C13-phase-cal_14_ch1.wfmx',
 'C13-phase-cal_15_ch1.wfmx',
 'C13-phase-cal_16_ch1.wfmx',
 'C13-phase-cal_17_ch1.wfmx',
 'C13-phase-cal_18_ch1.wfmx',
 'C13-phase-cal_19_ch1.wfmx',
 'C13-phase-cal_1_ch1.wfmx',
 'C13-phase-cal_20_ch1.wfmx',
 'C13-phase-cal_21_ch1.wfmx',
 'C13-phase-cal_22_ch1.wfmx',
 'C13-phase-cal_23_ch1.wfmx',
 'C13-phase-cal_24_ch1.wfmx',
 'C13-phase-cal_25_ch1.wfmx',
 'C13-phase-cal_26_ch1.wfmx',
 'C13-phase-cal_27_ch1.wfmx',
 'C13-phase-cal_28_ch1.wfmx',
 'C13-phase-cal_29_ch1.wfmx',
 'C13-phase-cal_2_ch1.wfmx',
 'C13-phase-cal_30_ch1.wfmx',
 'C13-phase-cal_31_ch1.wfmx',
 'C13-phase-cal_32_ch1.wfmx',
 'C13-phase-cal_33_ch1.wfmx',
 'C13-phase-cal_34_ch1.wfmx',
 'C13-phase-cal_35_ch1.wfmx',
 'C13-phase-cal_36_ch1.wfmx',
 'C13-phase-cal_37_ch1.wfmx',
 'C13-phase-cal_38_ch1.wfmx',
 'C13-phase-cal_39_c

In [36]:
awg.read_segment_table(1)

AttributeError: 'AWG70K' object has no attribute 'read_segment_table'